In [1]:
import os
import torch
from torch import nn
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from clipseg_model import CLIPSeg
from data.cub_parts_dataset import CUBPartsDataset, collate_fn
from data.cub_dataset import build_datasets, collate_fn

%load_ext autoreload
%autoreload 2

model = CLIPSeg()
state_dict = torch.load('checkpoints/clipseg_pascub_ft.pt', map_location='cpu')
model.load_state_dict(state_dict)

Some weights of CLIPSegForImageSegmentation were not initialized from the model checkpoint at CIDAS/clipseg-rd64-refined and are newly initialized: ['non_object_embedding', 'text_adapter.fc.0.weight', 'text_adapter.fc.2.weight', 'tunable_linear.weight', 'visual_adapter.fc.0.weight', 'visual_adapter.fc.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'clipseg.configuration_clipseg.CLIPSegConfig'> 512


<All keys matched successfully>

In [2]:
# dataset_test = CUBPartsDataset('datasets/PASCUB', split='test')
# dataloader_test = DataLoader(dataset=dataset_test, batch_size=2, shuffle=False, collate_fn=collate_fn)
# dataloader_test_iter = iter(dataloader_test)

(dataset_train, dataset_val, dataset_test), attr_indices, class_attrs_df = build_datasets(
    dataset_dir=os.path.join('datasets', 'CUB'),
    attr_subset='cbm',
    use_class_level_attr=True,
    use_transforms=False
)

dataloader_train = DataLoader(dataset_train, collate_fn=collate_fn, batch_size=4, shuffle=True)
dataloader_train_iter = iter(dataloader_train)

In [3]:
batch = next(dataloader_train_iter)
im_ids, imgs, class_gts, contrastive_gts = batch
contrastive_gts = contrastive_gts.to('cuda')
model.train()
with torch.no_grad():
    model(imgs, contrastive_gts)

torch.Size([32, 485, 64]) torch.Size([232, 485, 64])
torch.Size([4, 7, 64]) torch.Size([4, 57, 64])
torch.float32
torch.Size([4, 7, 57]) torch.Size([4, 7, 57])


In [4]:
F.cosine_similarity(torch.tensor([[1,2,3]], dtype=torch.float32), torch.tensor([[1,2,3]], dtype=torch.float32))

tensor([1.0000])

In [5]:
a = torch.stack([torch.full((485, 768), i) for i in range(4)])
a = a.unsqueeze(1).repeat(1, 8, 1, 1).reshape(-1, 485, 768)
a = a.reshape(4, 8, 485, 768)
a[1, ...].max()

tensor(1)